In [1]:
#begin by importing your lbiraries 
import pandas as pd
import numpy as np 
import csv
import requests
from bs4 import BeautifulSoup  #Beautifulsoup is used to separate out the HTML
get_ipython().system('conda install beautifulsoup4')

Solving environment: done

# All requested packages already installed.



In [2]:
pd.set_option('max_colwidth', 800) 


# Preprocessing and table extraction from Wikipedia

In [3]:
#This is a request to pull the wikipedia page in. 
Page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 


In [4]:
# getting data from internet
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text
#using the soup library
soup = BeautifulSoup(raw_wikipedia_page,'xml')



In [8]:
# Creating our Table
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []



#Make sure our table is clean when we pull it from wikipedia
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
Postcode.append(Postcode_var)
Borough.append(Borough_var)
Neighbourhood.append(Neighbourhood_var)

In [11]:
#Organizing multiple neighborhood postal codes 
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 77


# Identifying unique postal codes from the table

In [14]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M9M,North York,"Emery, Humberlea"
1,M1S,Scarborough,Agincourt
2,M3H,North York,"Bathurst Manor, Wilson Heights"
3,M3J,North York,"Northwood Park, York University"
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"


In [15]:
df_toronto.shape


(77, 3)

# Part Two: Bringing in the lat, long

In [23]:
#Install folium and import library for our next section
!pip install folium

In [26]:
import folium
import json, lxml
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [33]:
df = pd.read_csv (r'http://cocl.us/Geospatial_data')
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#We rename the column name for toronto to match the other table
df_toronto.rename(columns = {"Postcode": "Postal Code"}, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M9M,North York,"Emery, Humberlea"
1,M1S,Scarborough,Agincourt
2,M3H,North York,"Bathurst Manor, Wilson Heights"
3,M3J,North York,"Northwood Park, York University"
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"


In [38]:
#Merge the tables on postal code and create a new dataframe 
canada_df = pd.merge(df_toronto, df, on = "Postal Code")
canada_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M3H,North York,"Bathurst Manor, Wilson Heights",43.754328,-79.442259
3,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
4,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
5,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford Heights",43.757410,-79.273304
6,M6K,West Toronto,"Exhibition Place, Parkdale Village",43.636847,-79.428191
7,M4G,East York,Leaside,43.709060,-79.363452
8,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, South Niagara",43.628947,-79.394420
